In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
from car_detector.hybridnets import HybridNets, optimized_model
from car_detector.hybridnets.utils import util_draw_bird_eye_view, get_horizon_points
from collections import OrderedDict

In [2]:
VIDEO_FILE = "demo_video/snake.mp4"
model_path = "car_detector/models/hybridnets_384x512/hybridnets_384x512.onnx"
anchor_path = "car_detector/models/hybridnets_384x512/anchors_384x512.npy"

# Initialize road detector
optimized_model(model_path) # Remove unused nodes
car_detector = HybridNets(model_path, anchor_path, conf_thres=0.5, iou_thres=0.5)

/home/chienfeng/.virtualenvs/hackthon/lib/python3.7/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:56: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names))


In [3]:
tracker = IOUTracker(max_lost=5, iou_threshold=0.5, min_detection_confidence=0.4, max_detection_confidence=0.7,
                         tracker_output_format='mot_challenge')

In [4]:
def insert(dict, tracks):
    for track in tracks:
        frame, id, bb_left, bb_top, bb_right, bb_bottom, confidence, x, y, z = track
        frame -= 1
        if dict.get(id) is not None:
            dict[id].append([frame, bb_left, bb_top, bb_right, bb_bottom])
        else:
            dict[id] = [frame, bb_left, bb_top, bb_right, bb_bottom]


In [7]:
track_dict = {}
cap = cv.VideoCapture(VIDEO_FILE)
ok, image = cap.read()
H, W = int(image.shape[0]/2), int(image.shape[1]/2)
center = np.array([W/2, H/2]).astype(int)
line_indices = [np.arange(H/2,H).astype(int), 
                (W/2*np.ones_like(np.arange(H/2,H))).astype(int)]
prev_flag, now_flag = False, False
prev_frame, now_frame = 0, 0
danger = (0, 0)
cnt = 0

img_list = []
while True:
    ok, image = cap.read()

    if not ok:
        print("Cannot read the video feed.")
        break
    
    now_frame += 1
    image = cv.resize(image, (W, H))
    seg_map, filtered_boxes, filtered_scores = car_detector(image)

    class_ids = np.ones_like(filtered_scores).astype(int)
    confidence = np.ones_like(filtered_scores)
    tracks = tracker.update(filtered_boxes, confidence, class_ids=class_ids)

    updated_image = car_detector.draw_2D(image, alpha=0)
    
    # detect snake
    if (updated_image[center[1], center[0], :] == [192, 67, 251]).all():
        updated_image[line_indices[0], line_indices[1], :] = [0, 0, 255]
        now_flag = True
        if bool(now_flag) ^ bool(prev_flag):
            cnt += 1
            if cnt == 4:
                grad = cnt/(now_frame - prev_frame)
                if grad > (4/300):
                    print('Dangerous!')
                    danger = (now_frame, now_frame + 30)
                prev_frame = now_frame
                cnt = 0
        prev_flag = now_flag
    else:
        now_flag = False
        if bool(now_flag) ^ bool(prev_flag):
            cnt += 1
            if cnt == 4:
                grad = cnt/(now_frame - prev_frame)
                if grad > (4/300):
                    print('Dangerous!')
                    danger = (now_frame, now_frame + 30)
                prev_frame = now_frame
                cnt = 0
        prev_flag = now_flag
        updated_image[line_indices[0], line_indices[1], :] = [124, 252, 0]
    
    if now_frame > danger[0] and now_frame < danger[1]:
        updated_image = cv.putText(updated_image, 'Dangerous!', (center[0], 50), cv.FONT_HERSHEY_SIMPLEX, 1
                   , (0, 0, 255), 2, cv.LINE_AA)
    
    img_list.append(cv.cvtColor(updated_image, cv.COLOR_BGR2RGB))
    cv.imshow("image", updated_image)
    if cv.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Dangerous!
Dangerous!
Cannot read the video feed.


In [8]:
import imageio
imageio.mimsave(f"self_snake.gif", img_list, duration=0.15)